In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (./) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 🌍 Global Earthquake Events — Exploratory Data Analysis
**Dataset:** Global Earthquake Events (Kaggle — zahranusratt)  
**Coverage:** July–August 2016 | ~8,400 seismic events | 118 unique regions

---


In [ ]:
# ── Imports ──────────────────────────────────────────────────────────
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns
from datetime import datetime

sns.set_theme(style="whitegrid", palette="mako")
plt.rcParams.update({
    "figure.dpi": 120,
    "axes.titlesize": 13,
    "axes.labelsize": 11,
    "figure.facecolor": "white",
})

# ── Load & Preview ──────────────────────────────────────────────────
df = pd.read_csv("/kaggle/input/global-earthquake-events-dataset/earthquakes.csv")


# Parse the full timestamp
df["datetime"] = pd.to_datetime(df["time.full"])
df["date"] = df["datetime"].dt.date

# Rename columns for convenience
df.rename(columns={
    "impact.gap": "gap",
    "impact.magnitude": "magnitude",
    "impact.significance": "significance",
    "location.depth": "depth_km",
    "location.distance": "distance",
    "location.full": "full_location",
    "location.latitude": "lat",
    "location.longitude": "lon",
    "location.name": "region",
    "time.hour": "hour",
    "time.month": "month",
    "time.day": "day",
}, inplace=True)

print(f"Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Date range: {df['datetime'].min()} → {df['datetime'].max()}")
print(f"Regions: {df['region'].nunique()}")
print(f"Missing values: {df.isnull().sum().sum()}")
print()
df.head(3)


In [ ]:
# ── Statistical Summary ──────────────────────────────────────────────
key_cols = ["magnitude", "depth_km", "significance", "gap", "distance"]
summary = df[key_cols].describe().T
summary["skew"] = df[key_cols].skew()
summary["kurtosis"] = df[key_cols].kurtosis()
summary.round(3)


In [ ]:
# ── Distribution Plots ───────────────────────────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(16, 9))

# 1 – Magnitude histogram
ax = axes[0, 0]
ax.hist(df["magnitude"], bins=60, color="#2d6a4f", edgecolor="white", linewidth=0.3)
ax.set_title("Magnitude Distribution")
ax.set_xlabel("Magnitude")
ax.axvline(df["magnitude"].median(), color="tomato", ls="--", lw=1.2, label=f'Median {df["magnitude"].median():.2f}')
ax.legend(fontsize=9)

# 2 – Depth histogram (log-scale x)
ax = axes[0, 1]
ax.hist(df["depth_km"], bins=80, color="#1b4332", edgecolor="white", linewidth=0.3)
ax.set_title("Depth Distribution (km)")
ax.set_xlabel("Depth (km)")

# 3 – Significance histogram
ax = axes[0, 2]
ax.hist(df["significance"], bins=60, color="#40916c", edgecolor="white", linewidth=0.3)
ax.set_title("Significance Score Distribution")
ax.set_xlabel("Significance")

# 4 – Magnitude by top 10 regions (boxplot)
ax = axes[1, 0]
top10 = df["region"].value_counts().head(10).index
sub = df[df["region"].isin(top10)]
order = sub.groupby("region")["magnitude"].median().sort_values(ascending=False).index
sns.boxplot(data=sub, y="region", x="magnitude", order=order, ax=ax,
            palette="YlGn_r", linewidth=0.7, fliersize=2)
ax.set_title("Magnitude by Top 10 Regions")
ax.set_xlabel("Magnitude")
ax.set_ylabel("")

# 5 – Events per day (time series)
ax = axes[1, 1]
daily = df.groupby("date").size()
ax.fill_between(daily.index, daily.values, alpha=0.45, color="#2d6a4f")
ax.plot(daily.index, daily.values, color="#1b4332", linewidth=0.8)
ax.set_title("Daily Earthquake Count")
ax.set_xlabel("Date")
ax.tick_params(axis="x", rotation=45)

# 6 – Hourly distribution
ax = axes[1, 2]
hourly = df["hour"].value_counts().sort_index()
ax.bar(hourly.index, hourly.values, color="#52b788", edgecolor="white", linewidth=0.4)
ax.set_title("Events by Hour of Day (UTC)")
ax.set_xlabel("Hour")
ax.set_xticks(range(0, 24, 3))

plt.tight_layout()
plt.savefig("eda_distributions.png", dpi=150, bbox_inches="tight")
plt.show()


In [ ]:
# ── Relationships & Correlations ─────────────────────────────────────
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# 1 – Correlation heatmap
ax = axes[0]
corr = df[["magnitude", "depth_km", "significance", "gap", "distance"]].corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", cmap="YlGn",
            linewidths=0.5, ax=ax, vmin=-1, vmax=1, square=True)
ax.set_title("Feature Correlation Matrix")

# 2 – Magnitude vs Depth scatter
ax = axes[1]
scatter = ax.scatter(df["depth_km"], df["magnitude"], c=df["significance"],
                     cmap="YlOrRd", s=8, alpha=0.5, edgecolors="none")
ax.set_title("Magnitude vs Depth")
ax.set_xlabel("Depth (km)")
ax.set_ylabel("Magnitude")
plt.colorbar(scatter, ax=ax, label="Significance", shrink=0.8)

# 3 – Magnitude vs Significance
ax = axes[2]
ax.scatter(df["magnitude"], df["significance"], s=8, alpha=0.4, color="#2d6a4f")
ax.set_title("Magnitude vs Significance")
ax.set_xlabel("Magnitude")
ax.set_ylabel("Significance")

plt.tight_layout()
plt.savefig("eda_correlations.png", dpi=150, bbox_inches="tight")
plt.show()


In [ ]:
# ── Geographic Scatter (basic world view) ────────────────────────────
fig, ax = plt.subplots(figsize=(16, 8))
scatter = ax.scatter(
    df["lon"], df["lat"],
    c=df["magnitude"], cmap="YlOrRd",
    s=df["magnitude"] ** 2 * 2,
    alpha=0.5, edgecolors="gray", linewidth=0.2
)
ax.set_xlim(-180, 180)
ax.set_ylim(-90, 90)
ax.set_title("Global Earthquake Locations — sized & colored by magnitude", fontsize=14)
ax.set_xlabel("Longitude")
ax.set_ylabel("Latitude")
ax.set_facecolor("#eaf4f4")
plt.colorbar(scatter, label="Magnitude", shrink=0.6, pad=0.02)
plt.tight_layout()
plt.savefig("eda_geomap.png", dpi=150, bbox_inches="tight")
plt.show()


## 🔥 Interactive Heatmap — Global Seismic Activity
Dark base map with a heat layer weighted by magnitude. Toggle the **Significant Quakes (≥ 4.5)** layer to see clickable markers with event details.

In [ ]:
# ── Interactive Folium Heatmap ────────────────────────────────────────
import folium
from folium.plugins import HeatMap, MarkerCluster

# --- Base map ---
m = folium.Map(location=[20, 0], zoom_start=2, tiles="CartoDB dark_matter",
               width="100%", height="100%")

# --- Heatmap layer (all events, weighted by magnitude) ---
heat_data = df[["lat", "lon", "magnitude"]].dropna().values.tolist()
HeatMap(
    heat_data,
    radius=10,
    blur=12,
    max_zoom=6,
    gradient={0.2: "#ffffb2", 0.4: "#fecc5c", 0.6: "#fd8d3c", 0.8: "#f03b20", 1.0: "#bd0026"},
    name="Heatmap (all events)"
).add_to(m)

# --- Significant quakes as circle markers (magnitude ≥ 4.5) ---
sig_group = folium.FeatureGroup(name="Significant Quakes (≥ 4.5)")
for _, row in df[df["magnitude"] >= 4.5].iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=max(row["magnitude"] * 2, 4),
        color="#ff4444",
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(
            f"<b>{row['full_location']}</b><br>"
            f"Mag: {row['magnitude']}<br>"
            f"Depth: {row['depth_km']} km<br>"
            f"Date: {row['datetime'].strftime('%Y-%m-%d %H:%M')}",
            max_width=250
        ),
        tooltip=f"M{row['magnitude']} — {row['region']}"
    ).add_to(sig_group)
sig_group.add_to(m)

# --- Layer control ---
folium.LayerControl(collapsed=False).add_to(m)

# Save as standalone HTML & display inline
m.save("earthquake_heatmap.html")
print(f"✅ Interactive map saved → earthquake_heatmap.html")
print(f"   Heatmap: {len(heat_data):,} events | Markers: {len(df[df['magnitude'] >= 4.5]):,} significant quakes")
m


In [ ]:
# ── Top 20 Regions by Event Count ────────────────────────────────────
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Count
ax = axes[0]
top20 = df["region"].value_counts().head(20)
bars = ax.barh(top20.index[::-1], top20.values[::-1], color="#40916c", edgecolor="white")
ax.set_title("Top 20 Regions by Earthquake Count")
ax.set_xlabel("Number of Events")
for bar in bars:
    ax.text(bar.get_width() + 15, bar.get_y() + bar.get_height()/2,
            f'{int(bar.get_width())}', va='center', fontsize=8)

# Mean magnitude
ax = axes[1]
top20_mag = df.groupby("region")["magnitude"].agg(["mean", "count"])
top20_mag = top20_mag[top20_mag["count"] >= 10].sort_values("mean", ascending=True).tail(20)
ax.barh(top20_mag.index, top20_mag["mean"], color="#d4a373", edgecolor="white")
ax.set_title("Top 20 Regions by Mean Magnitude (min 10 events)")
ax.set_xlabel("Mean Magnitude")

plt.tight_layout()
plt.savefig("eda_regions.png", dpi=150, bbox_inches="tight")
plt.show()


In [ ]:
# ── Key EDA Findings ─────────────────────────────────────────────────
sig_quakes = df[df["magnitude"] >= 5.0]
print("=" * 60)
print("KEY FINDINGS")
print("=" * 60)
print(f"Total events: {len(df):,}")
print(f"Date range: {df['datetime'].min().date()} to {df['datetime'].max().date()}")
print(f"Magnitude range: {df['magnitude'].min()} – {df['magnitude'].max()}")
print(f"Mean magnitude: {df['magnitude'].mean():.2f} | Median: {df['magnitude'].median():.2f}")
print(f"Mean depth: {df['depth_km'].mean():.1f} km | Max: {df['depth_km'].max():.1f} km")
print(f"Events ≥ 5.0 magnitude: {len(sig_quakes)} ({len(sig_quakes)/len(df)*100:.1f}%)")
print(f"Events ≥ 6.0 magnitude: {len(df[df['magnitude'] >= 6.0])}")
print(f"Top 3 regions: {', '.join(df['region'].value_counts().head(3).index)}")
print(f"Busiest day: {daily.idxmax()} ({daily.max()} events)")
print()
print("Strongest earthquakes:")
print(df.nlargest(5, "magnitude")[["datetime", "full_location", "magnitude", "depth_km", "significance"]].to_string(index=False))
